In [1]:
!pip install dgl
from os import listdir
import dgl
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

     |████████████████████████████████| 3.0MB 3.5MB/s 


DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
filename = "/content/drive/My Drive/adaptec1.nets"

edge_file = "/content/add_edges.py"
edge_file_handle = open(edge_file, "w+")

node_count = 0

with open(filename,encoding="utf8") as f:
    netSection = False
    edges = 0
    lineCount = 0
    src=[]
    dest=[]
    for line in f:
        lineCount += 1
        #Newline
        if len(line) < 2:
            continue
            
        # Comments
        if line[0] == '#':
            continue
        #print(line)
        splitLine = line.split()
        #print("Words = ", len(splitLine))
        if(splitLine[0] == 'NetDegree'):
            
            if edges > 0: #Previous NetDegree not complete!
                print("Missed an edge at line: ", lineCount-1)
                
            edges = int(splitLine[2])
            netSection = True
            src=[]
            dest=[]
            #G.add_nodes(num)
            
        elif netSection == True:
          node_name = splitLine[0][1:]
          node_val  = int(node_name)
          if node_count < node_val+1:
              node_count = node_val+1
          
          if(splitLine[1] == 'O'):
              src.append(node_name)
              edges -= 1
          elif(splitLine[1] == 'I'):
              dest.append(node_name)
              edges -= 1
      
        if netSection == True:
            if edges == 0:
                if (len(src)<1):
                    print("No source edge, skipping")
                    continue
                if (len(dest)<1):
                    print("No dest edge, skipping")
                    continue
                    
                netSection = False
                for des_idx in range(len(dest)):
                    cmd = "G.add_edges("+src[0]+","+dest[des_idx]+")"+"\n"
                    edge_file_handle.write(cmd)
                    
        #if lineCount > 100:
        #    break


edge_file_handle.close()


No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584552
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584660
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584723
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584753
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584781
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584978
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  585078
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  585138
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  585199
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  585306
No dest edge, skipping
No dest edge, skipping
Missed an edge at line: 

In [4]:
G = dgl.DGLGraph()
G.add_nodes(node_count)
exec(open(edge_file).read())
print("Nodes = ", G.number_of_nodes())
print("Edges = ", G.number_of_edges())

Nodes =  211447
Edges =  720134


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
embed = nn.Embedding(G.number_of_nodes(),6)

In [0]:
G.ndata['feat'] = embed.weight

In [0]:
from dgl.nn.pytorch import GraphConv

In [0]:
class GCN(nn.Module):
    def __init__(self,input_size,hidden_size1,hidden_size2,num_classes):
        super(GCN,self).__init__()
        self.conv1 = GraphConv(input_size,hidden_size1)
        self.conv2 = GraphConv(hidden_size1,hidden_size2)
        self.conv3 = GraphConv(hidden_size2,num_classes)
    
    def forward(self,G,inputs):
        h = self.conv1(G,inputs)
        h = torch.relu(h)
        h = self.conv2(G,h)
        h = torch.relu(h)
        h = self.conv3(G,h)
        return h

In [0]:
net = GCN(6,5,4,20)

In [0]:
inputs = embed.weight
a = np.linspace(0,G.number_of_nodes()-1,20,dtype=int)
labelled_nodes = torch.tensor(a)
labels = torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])

# Input from user hyperparameters: 
- Number of epochs to train
- Learning rate

In [15]:
epochs = int(input("Enter number of epochs: "))
learning_rate = float(input("Enter learning rate: "))

Enter number of epochs: 250
Enter learning rate: 0.01


# Training Model

In [16]:
import itertools
import time

optimizer = torch.optim.Adam(itertools.chain(net.parameters(), embed.parameters()), lr=learning_rate)
all_logits = []

start_time = time.time()

for epoch in range(epochs):
    logits = net(G, inputs)
    all_logits.append(logits.detach())
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[labelled_nodes], labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print('Epoch {} | Loss: {}'.format(epoch + 1, loss.item()))

end_time = time.time()
print("Training Time: {}".format(end_time - start_time))

Epoch 1 | Loss: 3.0097107887268066
Epoch 2 | Loss: 2.99682354927063
Epoch 3 | Loss: 2.985320568084717
Epoch 4 | Loss: 2.9746124744415283
Epoch 5 | Loss: 2.9643683433532715
Epoch 6 | Loss: 2.954580783843994
Epoch 7 | Loss: 2.9448184967041016
Epoch 8 | Loss: 2.9347147941589355
Epoch 9 | Loss: 2.924044132232666
Epoch 10 | Loss: 2.9129624366760254
Epoch 11 | Loss: 2.9014732837677
Epoch 12 | Loss: 2.8892927169799805
Epoch 13 | Loss: 2.876319408416748
Epoch 14 | Loss: 2.8623850345611572
Epoch 15 | Loss: 2.8474600315093994
Epoch 16 | Loss: 2.8315606117248535
Epoch 17 | Loss: 2.814587116241455
Epoch 18 | Loss: 2.7965919971466064
Epoch 19 | Loss: 2.7775731086730957
Epoch 20 | Loss: 2.757457971572876
Epoch 21 | Loss: 2.7367522716522217
Epoch 22 | Loss: 2.715223789215088
Epoch 23 | Loss: 2.6929526329040527
Epoch 24 | Loss: 2.670043468475342
Epoch 25 | Loss: 2.6462440490722656
Epoch 26 | Loss: 2.6216001510620117
Epoch 27 | Loss: 2.596569776535034
Epoch 28 | Loss: 2.5711426734924316
Epoch 29 | Loss

In [0]:
pos = {}
cluster_nodes = {} # Every value in key->value pair includes the cluster centre node as well.
for v in range(G.number_of_nodes()):
  pos[v] = all_logits[99][v].numpy()
  cls = pos[v].argmax()
  if cluster_nodes.get(cls) :
    cluster_nodes[cls].append(v)
  else:
    cluster_nodes[cls] = []
    cluster_nodes[cls].append(v)

In [18]:
print(cluster_nodes)

{0: [0, 3, 4, 163, 456, 674, 852, 980, 1009, 1095, 1201, 1236, 1250, 1515, 1718, 2114, 2122, 2189, 2191, 2233, 2255, 2322, 2541, 2543, 2631, 3073, 3267, 3420, 3432, 3433, 3451, 3461, 3510, 4209, 4279, 4331, 4631, 4641, 4697, 4724, 4732, 4790, 4808, 4816, 4818, 4910, 4957, 4981, 5013, 5018, 5021, 5046, 5073, 5074, 5080, 5174, 5217, 5226, 5238, 5244, 5266, 5274, 5282, 5323, 5386, 5507, 5511, 5580, 5587, 5627, 5703, 5705, 5852, 5869, 6028, 6180, 6528, 6727, 6728, 6730, 7264, 7269, 7456, 7462, 7508, 7597, 7609, 7921, 8473, 8540, 8557, 8560, 8631, 8667, 8837, 8944, 9279, 9286, 9290, 9342, 9355, 9415, 9421, 9685, 9755, 9955, 10448, 10523, 10531, 10545, 10547, 10556, 10597, 10616, 10625, 10715, 10939, 10954, 10960, 10964, 11032, 11066, 11068, 11070, 11076, 11091, 11109, 11136, 11141, 11155, 11158, 11170, 11182, 11201, 11241, 11250, 11291, 11350, 11398, 11408, 11411, 11424, 11498, 11850, 11869, 11997, 12141, 12179, 12303, 12470, 12471, 12472, 12748, 12772, 12797, 12799, 12819, 12829, 12857, 12

# Cluster Quality Metric #2 : Number of Nodes per Cluster Distribution

In [0]:
def number_of_nodes_per_cluster():
  number_dict = {} # Dictionary mapping the node label to the number of nodes in a particular cluster
  for key,value in cluster_nodes.items():
    number_dict[key] = len(value)

  number_dict = sorted(number_dict.items()) # Sorting dictionary by keys in increasing order

  for ix in number_dict:
    print("Number of Nodes in Cluster {} = {}\n".format(ix[0],ix[1]))

  return number_dict

In [20]:
number_dict = number_of_nodes_per_cluster()

Number of Nodes in Cluster 0 = 2820

Number of Nodes in Cluster 1 = 13196

Number of Nodes in Cluster 2 = 29582

Number of Nodes in Cluster 3 = 9415

Number of Nodes in Cluster 4 = 9274

Number of Nodes in Cluster 5 = 78526

Number of Nodes in Cluster 6 = 39

Number of Nodes in Cluster 7 = 9213

Number of Nodes in Cluster 8 = 3360

Number of Nodes in Cluster 9 = 8667

Number of Nodes in Cluster 10 = 5998

Number of Nodes in Cluster 11 = 1876

Number of Nodes in Cluster 12 = 4399

Number of Nodes in Cluster 13 = 4276

Number of Nodes in Cluster 14 = 3885

Number of Nodes in Cluster 15 = 136

Number of Nodes in Cluster 16 = 4123

Number of Nodes in Cluster 17 = 10847

Number of Nodes in Cluster 18 = 1849

Number of Nodes in Cluster 19 = 9966



# HashMap (Dictionary) for easy visualization of networkx graph
- Dictionary to map node numbers in a particular cluster to a standard count procedure of 0 to (n-1) 
- n = number of nodes in a particular cluster
- Cluster label is taken as input from user

In [43]:
num = int(input("Enter cluster number: "))

Enter cluster number: 10


In [0]:
standard_nodes_dict = {}

count = 0

for node in cluster_nodes[num]:
  standard_nodes_dict[node] = count
  count += 1

In [45]:
print(standard_nodes_dict.items())

dict_items([(42, 0), (43, 1), (320, 2), (321, 3), (322, 4), (328, 5), (329, 6), (445, 7), (521, 8), (522, 9), (523, 10), (524, 11), (525, 12), (526, 13), (536, 14), (537, 15), (538, 16), (542, 17), (543, 18), (544, 19), (575, 20), (576, 21), (577, 22), (623, 23), (640, 24), (652, 25), (657, 26), (662, 27), (702, 28), (707, 29), (792, 30), (796, 31), (798, 32), (799, 33), (802, 34), (805, 35), (806, 36), (923, 37), (941, 38), (943, 39), (962, 40), (1032, 41), (1033, 42), (1101, 43), (1138, 44), (1210, 45), (1281, 46), (1305, 47), (1306, 48), (1317, 49), (1318, 50), (1333, 51), (1334, 52), (1391, 53), (1416, 54), (1418, 55), (1427, 56), (1459, 57), (1558, 58), (1611, 59), (1613, 60), (1637, 61), (1667, 62), (1689, 63), (1715, 64), (1716, 65), (1729, 66), (1759, 67), (1775, 68), (1780, 69), (1794, 70), (1795, 71), (1804, 72), (1845, 73), (1849, 74), (1882, 75), (1897, 76), (1908, 77), (1911, 78), (1927, 79), (1943, 80), (1960, 81), (1972, 82), (1975, 83), (1976, 84), (1992, 85), (2090, 86

# Clustering Visualisation: Cluster Quality Metric # 3
- Visualisation for any generic cluster label taken as input from user

In [0]:
# dict_keys = list(standard_nodes_dict.keys())

# input_edges = G.in_edges(dict_keys, form='uv')

# ax = input_edges[0].numpy()

# ax = [i for i in ax if i in standard_nodes_dict.keys()]

# print(ax)

In [0]:
src = []
dst = []

In [0]:
def build_graph():

  dict_keys = list(standard_nodes_dict.keys())

  input_edges = G.in_edges(dict_keys, form='uv')

  src = input_edges[0].numpy()
  dst = input_edges[1].numpy()

  m = src.shape[0]
  src1 = []
  dst1 = []

  for ix in range(m):
    if src[ix] not in standard_nodes_dict.keys():
      continue
    else:
      src1.append(src[ix])
      dst1.append(dst[ix])

  src1 = np.array(src1)
  dst1 = np.array(dst1)

  j = 0
  for num in src1:
    num = standard_nodes_dict.get(num)
    src1[j] = num
    j += 1

  i = 0
  for num1 in dst1:
      num1 = standard_nodes_dict[num1]
      dst1[i] = num1
      i += 1

  u = np.concatenate([src1,dst1])
  v = np.concatenate([dst1,src1])

  return dgl.DGLGraph((u,v))

In [0]:
G1 = build_graph()

In [50]:
print("Nodes = {}".format(G1.number_of_nodes()))
print("Edges = {}".format(G1.number_of_edges()))

Nodes = 5997
Edges = 3094


In [0]:
nx_G1 = G1.to_networkx().to_undirected()

In [0]:
nx.write_edgelist(nx_G1,"/content/cluster10.csv")

In [0]:
# pos1 = nx.kamada_kawai_layout(nx_G1) # Session crashes on running this cell

In [0]:
# plt.style.use("seaborn")

In [0]:
# nx.draw(nx_G1,pos1,with_labels=True,node_color=[[0.7,0.7,0.7]])
# plt.show()

# Cluster Quality Metric # 1: Total number of edges between nodes which belong to different clusters divided by total edges represented as percentage (%)

In [52]:
m = 10 # Total number of clusters

edges_between_clusters = {}

for i in range(m):
  print("Calculating for cluster {}\n".format(i))
  nodes_i = cluster_nodes[i]

  for j in range(i+1,m):
    nodes_j = np.array(cluster_nodes[j])

    ij_input_edges = G.in_edges(nodes_i,form='uv')
    edges_src = np.array(ij_input_edges[0])
    
    count = 0

    for val1 in nodes_j:
      for val2 in edges_src:
        if val1 == val2:
          count += 1

    # for x in range(np.array(nodes_i).shape[0]):
    #   if edges_src[x] == nodes_j[x]:
    #     count += 1

    edges_between_clusters[(i,j)] = count

Calculating for cluster 0

Calculating for cluster 1

Calculating for cluster 2

Calculating for cluster 3

Calculating for cluster 4

Calculating for cluster 5

Calculating for cluster 6

Calculating for cluster 7

Calculating for cluster 8

Calculating for cluster 9



In [53]:
edges_between_clusters

{(0, 1): 694,
 (0, 2): 1202,
 (0, 3): 526,
 (0, 4): 602,
 (0, 5): 2157,
 (0, 6): 4,
 (0, 7): 581,
 (0, 8): 187,
 (0, 9): 247,
 (1, 2): 4864,
 (1, 3): 1257,
 (1, 4): 2823,
 (1, 5): 10041,
 (1, 6): 4,
 (1, 7): 1894,
 (1, 8): 839,
 (1, 9): 1154,
 (2, 3): 5323,
 (2, 4): 12495,
 (2, 5): 37502,
 (2, 6): 2,
 (2, 7): 4098,
 (2, 8): 1606,
 (2, 9): 7463,
 (3, 4): 2174,
 (3, 5): 7536,
 (3, 6): 4,
 (3, 7): 1298,
 (3, 8): 363,
 (3, 9): 833,
 (4, 5): 8216,
 (4, 6): 12,
 (4, 7): 156,
 (4, 8): 83,
 (4, 9): 1532,
 (5, 6): 80,
 (5, 7): 5627,
 (5, 8): 1857,
 (5, 9): 20087,
 (6, 7): 0,
 (6, 8): 0,
 (6, 9): 4,
 (7, 8): 772,
 (7, 9): 1187,
 (8, 9): 634}

In [0]:
percentage_dict = {}

for key,value in edges_between_clusters.items():
  value = (value/G.number_of_edges())*100
  value = round(value,4)
  percentage_dict[key] = value

In [55]:
percentage_dict

{(0, 1): 0.0964,
 (0, 2): 0.1669,
 (0, 3): 0.073,
 (0, 4): 0.0836,
 (0, 5): 0.2995,
 (0, 6): 0.0006,
 (0, 7): 0.0807,
 (0, 8): 0.026,
 (0, 9): 0.0343,
 (1, 2): 0.6754,
 (1, 3): 0.1746,
 (1, 4): 0.392,
 (1, 5): 1.3943,
 (1, 6): 0.0006,
 (1, 7): 0.263,
 (1, 8): 0.1165,
 (1, 9): 0.1602,
 (2, 3): 0.7392,
 (2, 4): 1.7351,
 (2, 5): 5.2076,
 (2, 6): 0.0003,
 (2, 7): 0.5691,
 (2, 8): 0.223,
 (2, 9): 1.0363,
 (3, 4): 0.3019,
 (3, 5): 1.0465,
 (3, 6): 0.0006,
 (3, 7): 0.1802,
 (3, 8): 0.0504,
 (3, 9): 0.1157,
 (4, 5): 1.1409,
 (4, 6): 0.0017,
 (4, 7): 0.0217,
 (4, 8): 0.0115,
 (4, 9): 0.2127,
 (5, 6): 0.0111,
 (5, 7): 0.7814,
 (5, 8): 0.2579,
 (5, 9): 2.7893,
 (6, 7): 0.0,
 (6, 8): 0.0,
 (6, 9): 0.0006,
 (7, 8): 0.1072,
 (7, 9): 0.1648,
 (8, 9): 0.088}